In [2]:
%pip install astropy

  Obtaining dependency information for astropy from https://files.pythonhosted.org/packages/8c/4d/67c092bf54ed3be87fbe3dc4f5589ea56a23757594e221be5afb0d67b99e/astropy-5.3.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 15.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.3 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from astropy.io import ascii
from scipy.optimize import root_scalar
from astropy import units as u
from astropy import constants as const
from astropy.time import Time
import matplotlib.pyplot as plt
#ecuacion de kepler
def Kepler(E,l,ecc):
    f = E - ecc*np.sin(E)-l
    return f

#a = semieje mayor, e= excentricidad, omega= longitud del nodo ascendente, w = argumento del pericentro, 
#i = inclinacion de la orbita, n = movimiento medio, lo = anomalia media en la epoca, tp = tiempo a calcular 
#to = tiempo de la epoca, M = la suma de las masas de los cuerpos

#Orbita de la tierra
#Epoca
to = Time('2023-07-09T00:00:00', scale='tdb', format = 'isot')
#to = to.utc

#Efemerides Tierra
a = (1.495582533630905E+8*u.km).to('au')
ecc = 1.694863932474438E-02
omega = 1.498625963929686E+02*u.deg
w = 3.146587763491455E+02*u.deg
iota = 4.164075603038432E-03*u.deg 
n = 1.141204629731537E-05*u.deg/u.s
lo = 1.817846947871890E+02*u.deg

#Transformacion de unidades
iota = iota.to('rad')
omega = omega.to('rad')
w = w.to('rad')
n = n.to('rad/s')
lo = lo.to('rad')

#





<Quantity 7.26768296e-05 rad>